In [ ]:
# Import necessary libraries
import gymnasium as gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from datetime import datetime
import os
import random
import torch
import scipy.stats as stats
from scipy.ndimage import gaussian_filter1d
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
from tqdm.notebook import tqdm
import inspect
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import auc
from collections import deque

import sys
from pathlib import Path
save_path = "BipedalPerformanceResults"

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300

current_dir = os.getcwd()  
project_root = str(Path(current_dir).parent.parent)
sys.path.append(project_root)

# Load Project Modules
from AdaptiveRewardFunctionLearning.Prompts.prompts import device, apiKey, modelName
from RLEnvironment.env import CustomBipedalWalkerEnv
from AdaptiveRewardFunctionLearning.RewardGeneration.rewardCritic import RewardUpdateSystem
from RLEnvironment.training.agent import DQLearningAgent
from RLEnvironment.training.training import trainDQLearning

# Load BipedalWalker specific reward functions
from AdaptiveRewardFunctionLearning.RewardGeneration.bipedalRewardFunctions import (
    badRewardBipedal, 
    stabilityRewardBipedal,
    efficiencyRewardBipedal,
    potentialBasedRewardBipedal, 
    energyBasedRewardBipedal, 
    baselineRewardBipedal
)

In [ ]:
def safely_get_function_source(func):
    """
    Safely get the source code of a function, handling dynamically generated functions.
    Returns a string representation if source code cannot be retrieved.
    """
    # Handle the case when func is already a string
    if isinstance(func, str):
        return func
        
    try:
        return inspect.getsource(func)
    except (TypeError, OSError, IOError):
        # For dynamically generated functions, create a representation
        if hasattr(func, '__name__'):
            return f"<Dynamically generated function: {func.__name__}>"
        return f"<Dynamically generated function: {func}>"
        
def function_to_string(func):
    """
    Convert a function to a string representation, suitable for display and comparison.
    Handles various function types including lambdas, dynamically created functions, and strings.
    """
    if func is None:
        return "None"
    
    # Handle the case when func is already a string
    if isinstance(func, str):
        return func
        
    # Try to get the source code
    try:
        source = inspect.getsource(func)
        return source.strip()
    except (TypeError, OSError, IOError):
        # If we can't get the source, create a detailed representation
        if hasattr(func, '__name__'):
            func_name = func.__name__
        else:
            func_name = str(func)
            
        # Get signature if possible
        try:
            signature = str(inspect.signature(func))
        except (TypeError, ValueError):
            signature = "(unknown signature)"
            
        # For simple functions, we can create a representation of their logic
        try:
            # Try to get the code object
            code = func.__code__
            # Include variable names and constants from the function
            varnames = code.co_varnames[:code.co_argcount]
            constants = code.co_consts
            
            return f"Function {func_name}{signature} with {len(varnames)} arguments, {len(constants)} constants"
        except AttributeError:
            # Fallback for non-Python functions or built-ins
            return f"Function {func_name}{signature} (source unavailable)"

In [ ]:
def runBipedalPerformanceTest(
    episodes=5000, 
    changeInterval=1000, 
    legChanges=[0.3, 0.5],
    terrainChanges=[0.0, 1.0],
    seed=42,
    collect_component_data=True
):
    """Enhanced performance test for BipedalWalker with detailed adaptation metrics"""
    print(f"Starting BipedalWalker Performance Test with seed {seed}...")
    
    # Set all random seeds
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
    def create_fresh_env():
        # Create fresh environment with consistent settings
        env = gym.make('BipedalWalker-v3', render_mode=None)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        env.reset(seed=seed)
        env = CustomBipedalWalkerEnv(env, numComponents=2)
        env.setEnvironmentParameters(leg_length=legChanges[0], terrain_roughness=terrainChanges[0])
        return env
    
    def create_dqn_agent(env, device):
        return DQLearningAgent(
            env=env, 
            stateSize=env.observation_space.shape[0], 
            actionSize=env.action_space.n,
            device=device,
            learningRate=0.0005,      # Lower for continuous actions
            discountFactor=0.99,      # Standard value
            epsilon=1.0,
            epsilonDecay=0.9999,      # Slower decay for longer exploration
            epsilonMin=0.1,           # Higher minimum for exploration
            replayBufferSize=50000,   # Larger for complex environment
            batchSize=64,             # Larger batch for stability
            targetUpdateFreq=200      # Less frequent updates
        )
    
    # Create initial environment
    env = create_fresh_env()
    
    # Define reward function configurations
    rewardfunctions = {
        'adaptivereward': {
            'agent': None,  # Will be created fresh for each test
            'updatesystem': RewardUpdateSystem(apiKey, modelName),
            'rewardfunction': None,
            'update_method': 'llm'
        },
        'pbrs': {
            'agent': None,
            'updatesystem': None,
            'rewardfunction': potentialBasedRewardBipedal,  # Change to match import
            'update_method': None
        },
        'energy_based': {
            'agent': None,
            'updatesystem': None,
            'rewardfunction': energyBasedRewardBipedal,  # Change to match import
            'update_method': None
        },
        'baseline': {
            'agent': None,
            'updatesystem': None,
            'rewardfunction': baselineRewardBipedal,  # Change to match import
            'update_method': None
        }
    }

    results = {}
    test_order = ['adaptivereward', 'energy_based', 'baseline', 'pbrs']
    
    # Test each reward function
    for rewardname in test_order:
        print(f"\nTesting reward function: {rewardname}")
        
        # Create fresh environment and agent
        env = create_fresh_env()
        rewardfunctions[rewardname]['agent'] = create_dqn_agent(env, device)
        rewardinfo = rewardfunctions[rewardname]
        
        # Reset variables for this run
        currentLegIdx = 0
        currentTerrainIdx = 0
        
        # Set up the reward function
        if rewardname == 'adaptivereward':
            # Initialize both components for adaptive reward
            env.setComponentReward(1, stabilityRewardBipedal)
            env.setComponentReward(2, efficiencyRewardBipedal)
            rewardinfo['updatesystem'].lastUpdateEpisode = 0
        else:
            env.setRewardFunction(rewardinfo['rewardfunction'])
        
        # Storage for episode data
        episoderewards = []
        episodedistances = []
        rewardchangeepisodes = []
        environmentchanges = []
        
        # Enhanced metrics collection
        adaptation_metrics = {
            'pre_change_performance': [],  # Performance before env change
            'post_change_performance': [],  # Performance right after env change
            'recovery_times': [],           # Episodes to recover after change
            'performance_drop': [],         # Performance drop percentage
            'change_episodes': []           # Episode numbers where changes occurred
        }
        
        # Component weight tracking for adaptive reward only
        component_weights = [] if rewardname == 'adaptivereward' and collect_component_data else None
        component_updates = [] if rewardname == 'adaptivereward' and collect_component_data else None
        
        # Calculate rolling metrics function
        def calculate_rolling_metrics(episode_data, window=20):
            if len(episode_data) < window:
                return np.mean(episode_data) if episode_data else 0
            
            return np.mean(episode_data[-window:])
        
        def onEpisodeEnd(env, updatesystem, episode, reward, steps, info=None):
            nonlocal episoderewards, episodedistances, rewardchangeepisodes
            nonlocal environmentchanges, currentLegIdx, currentTerrainIdx
            nonlocal adaptation_metrics, component_weights, component_updates
            
            # Get distance traveled if available in info
            distance = info.get('distance', 0) if info else 0
            
            # Record basic metrics
            episoderewards.append(reward)
            episodedistances.append(distance)
            
            # Calculate rolling metrics for decision making
            metrics = {
                'currentEpisode': episode,
                'recentRewards': episoderewards[-100:] if len(episoderewards) > 100 else episoderewards,
                'averageDistance': np.mean(episodedistances[-100:]) if episodedistances else 0,
                'distanceVariance': np.var(episodedistances[-100:]) if len(episodedistances) > 1 else 0
            }
            
            # Collect component weights for adaptive reward
            if rewardname == 'adaptivereward' and collect_component_data and hasattr(env, 'getCurrentWeights'):
                weights = env.getCurrentWeights()
                component_weights.append({
                    'episode': episode,
                    'stability': weights['stability'],  # Use this key instead
                    'efficiency': weights['efficiency'] # Use this key instead
                })
            
            # Print debug info periodically
            if episode % 500 == 0:
                print(f"\nMetrics at Episode {episode}:")
                print(f"Recent Average Reward: {np.mean(metrics['recentRewards']):.2f}")
                print(f"Average Distance: {metrics['averageDistance']:.2f}")
                
                if rewardname == 'adaptivereward' and hasattr(env, 'getCurrentWeights'):
                    weights = env.getCurrentWeights()
                    print(f"Component Weights - Stability: {weights['stability']:.2f}, "
                        f"Efficiency: {weights['efficiency']:.2f}")
            
            # Handle LLM updates for adaptive reward
            if rewardname == 'adaptivereward' and updatesystem is not None:
                for component in range(1, 3):
                    updatesystem.targetComponent = component
                    if updatesystem.waitingTime(f'component_{component}', metrics, updatesystem.lastUpdateEpisode):
                        current_func = env.rewardComponents[f'rewardFunction{component}']
                        new_function, updated = updatesystem.validateAndUpdate(current_func)
                        
                        if updated:
                            if isinstance(new_function, str):
                                print("WARNING: New function returned as string, not function object")
                                continue  # Skip applying the update
                            else:
                                old_func_str = str(current_func)
                                new_func_str = str(new_function)
                            
                            # Apply the update
                            env.setComponentReward(component, new_function)
                            rewardchangeepisodes.append(episode)
                            updatesystem.lastUpdateEpisode = episode
                            
                            # Record the update details
                            if component_updates is not None:
                                component_updates.append({
                                    'episode': episode,
                                    'component': component,
                                    'old_function': old_func_str,
                                    'new_function': new_func_str,
                                    'pre_update_performance': calculate_rolling_metrics(episoderewards[-20:]) if len(episoderewards) >= 20 else 0
                                })
                            
                            print(f"✓ LLM update for component {component} at episode {episode}")
            
            # Environment changes for all approaches
            if episode % changeInterval == 0 and episode > 0:
                # Calculate pre-change performance (last 20 episodes)
                pre_change_perf = calculate_rolling_metrics(episoderewards[-20:])
                
                # Alternate between leg length and terrain roughness changes
                if episode % (changeInterval * 2) == 0:
                    # Update leg length
                    currentLegIdx = (currentLegIdx + 1) % len(legChanges)
                    new_leg_length = legChanges[currentLegIdx]
                    env.setEnvironmentParameters(leg_length=new_leg_length)
                    print(f"\nChanged leg length to: {new_leg_length} at episode {episode}")
                else:
                    # Update terrain roughness
                    currentTerrainIdx = (currentTerrainIdx + 1) % len(terrainChanges)
                    new_terrain = terrainChanges[currentTerrainIdx]
                    env.setEnvironmentParameters(terrain_roughness=new_terrain)
                    print(f"\nChanged terrain roughness to: {new_terrain} at episode {episode}")
                
                # Record that a change happened
                environmentchanges.append(episode)
                
                # Start tracking adaptation metrics for this change
                adaptation_metrics['pre_change_performance'].append(pre_change_perf)
                adaptation_metrics['change_episodes'].append(episode)
                
            # Track post-change performance and recovery
            if environmentchanges and (episode - environmentchanges[-1]) == 20:  # 20 episodes after change
                # Calculate performance drop
                post_change_perf = calculate_rolling_metrics(episoderewards[-20:])
                adaptation_metrics['post_change_performance'].append(post_change_perf)
                
                # Calculate performance drop as percentage
                pre_perf = adaptation_metrics['pre_change_performance'][-1]
                perf_drop_pct = (pre_perf - post_change_perf) / pre_perf * 100 if pre_perf > 0 else 0
                adaptation_metrics['performance_drop'].append(perf_drop_pct)
        
        # Train the agent
        agent, env, rewards = trainDQLearning(
            agent=rewardinfo['agent'],
            env=env,
            numEpisodes=episodes,
            updateSystem=rewardinfo['updatesystem'],
            onEpisodeEnd=onEpisodeEnd
        )
        
        # Calculate recovery times after training is complete
        for i, change_ep in enumerate(adaptation_metrics['change_episodes']):
            recovery_threshold = adaptation_metrics['pre_change_performance'][i] * 0.95
            recovery_time = episodes - change_ep  # Default: never recovered
            
            # Find first point after change where performance exceeds recovery threshold
            rolling_rewards = pd.Series(episoderewards[change_ep:]).rolling(window=20).mean()
            for j, val in enumerate(rolling_rewards):
                if not np.isnan(val) and val >= recovery_threshold:
                    recovery_time = j
                    break
                    
            adaptation_metrics['recovery_times'].append(recovery_time)
        
        # Store all results
        results[rewardname] = {
            'rewards': episoderewards,
            'distances': episodedistances,
            'rewardChanges': rewardchangeepisodes,
            'environmentChanges': environmentchanges,
            'adaptation_metrics': adaptation_metrics
        }
        
        # Add component data for adaptive reward
        if component_weights is not None:
            results[rewardname]['component_weights'] = component_weights
        
        if component_updates is not None:
            results[rewardname]['component_updates'] = component_updates
        
        # Print final metrics
        print(f"\nCompleted testing {rewardname}")
        print(f"Final average reward: {np.mean(episoderewards[-100:]):.2f}")
        print(f"Final average distance: {np.mean(episodedistances[-100:]):.2f}")
        if adaptation_metrics['recovery_times']:
            print(f"Average recovery time: {np.mean(adaptation_metrics['recovery_times']):.2f} episodes")
    
    return results

In [ ]:
def run_statistical_tests(results):
    """Run statistical tests to compare performance of different reward approaches"""
    # Dictionary to store test results
    test_results = {}
    
    # Prepare data for tests
    reward_data = []
    
    # Collect episode rewards for each approach
    for reward_type, reward_info in results.items():
        rewards = np.array(reward_info['rewards'])
        distances = np.array(reward_info['distances'])
        
        # Create a DataFrame row for each episode
        for i in range(len(rewards)):
            row = {
                'reward_type': reward_type,
                'episode': i,
                'reward': rewards[i],
                'distance': distances[i] if i < len(distances) else 0
            }
            reward_data.append(row)
    
    # Convert to DataFrame
    df = pd.DataFrame(reward_data)
    
    # Run tests for different metrics
    metrics = ['reward', 'distance']
    for metric in metrics:
        # Run ANOVA to test if there are significant differences between approaches
        formula = f"{metric} ~ C(reward_type)"
        model = ols(formula, data=df).fit()
        anova_table = anova_lm(model, typ=2)
        
        # Store ANOVA results
        p_value = anova_table['PR(>F)'][0]
        significant = p_value < 0.05
        
        # Print ANOVA results
        print(f"\nANOVA for {metric}:")
        print(anova_table)
        print(f"Significant difference: {significant} (p-value: {p_value:.4f})")
        
        # If ANOVA is significant, run pairwise t-tests with Bonferroni correction
        pairwise_results = {}
        if significant:
            # Get all unique reward types
            reward_types = df['reward_type'].unique()
            
            # Calculate number of comparisons for Bonferroni correction
            num_comparisons = len(reward_types) * (len(reward_types) - 1) // 2
            # Bonferroni-corrected alpha
            alpha_corrected = 0.05 / num_comparisons
            
            print(f"\nPairwise t-tests with Bonferroni correction (alpha = {alpha_corrected:.5f}):")
            
            # Run t-tests for each pair of reward types
            for i, type1 in enumerate(reward_types):
                for type2 in reward_types[i+1:]:
                    # Get data for each group
                    group1_data = df[df['reward_type'] == type1][metric]
                    group2_data = df[df['reward_type'] == type2][metric]
                    
                    # Run t-test
                    t_stat, p_val = stats.ttest_ind(group1_data, group2_data, equal_var=False)
                    
                    # Check if significant with Bonferroni correction
                    significant = p_val < alpha_corrected
                    
                    # Store results
                    pair = f"{type1} vs {type2}"
                    pairwise_results[pair] = {
                        'mean_diff': float(group1_data.mean() - group2_data.mean()),
                        'p_value': float(p_val),
                        'significant': bool(significant),
                        't_statistic': float(t_stat)
                    }
                    
                    # Print formatted results
                    sig_symbol = "*" if significant else ""
                    print(f"{pair}: diff = {pairwise_results[pair]['mean_diff']:.2f}, p = {p_val:.4f}{sig_symbol}")
        
        # Store all results for this metric
        test_results[metric] = {
            'anova_p_value': float(p_value),
            'anova_significant': bool(significant),
            'pairwise_comparisons': pairwise_results
        }
    
    return test_results

In [ ]:
def create_reward_over_time_plot(results, changeInterval, save_path=".", include_variance=True):
    """
    Create a reward over time plot with optional variance bands.
    Shows all reward functions on one plot with clear markers for environment changes.
    """
    plt.figure(figsize=(14, 8))
    
    # Define distinguishable colors and line styles
    colors = {
        'adaptivereward': '#1f77b4',  # blue
        'energy_based': '#2ca02c',    # green
        'pbrs': '#ff7f0e',            # orange
        'baseline': '#d62728'         # red
    }
    
    line_styles = {
        'adaptivereward': '-',
        'energy_based': '-.',
        'pbrs': '--',
        'baseline': ':'
    }
    
    # Plot each reward function
    for rewardname, rewardresults in results.items():
        rewards = rewardresults['rewards']
        
        # Apply smoother with larger window for cleaner visualization
        window = 50
        smoothed_rewards = pd.Series(rewards).rolling(window=window, min_periods=1).mean()
        
        if include_variance:
            # Calculate rolling standard deviation for variance bands
            rolling_std = pd.Series(rewards).rolling(window=window, min_periods=1).std()
            
            # Create confidence interval (1 standard deviation)
            upper_bound = smoothed_rewards + rolling_std
            lower_bound = smoothed_rewards - rolling_std
            
            # Fill the area between upper and lower bounds
            plt.fill_between(
                range(len(rewards)),
                lower_bound,
                upper_bound,
                alpha=0.2,
                color=colors.get(rewardname, 'black'),
                label=f"{rewardname} variance" if rewardname == list(results.keys())[0] else None
            )
        
        plt.plot(
            range(len(rewards)), 
            smoothed_rewards,
            label=rewardname,
            color=colors.get(rewardname, 'black'),
            linestyle=line_styles.get(rewardname, '-'),
            linewidth=2.5
        )
    
    # Add vertical lines for environment changes
    change_episodes = list(range(changeInterval, len(next(iter(results.values()))['rewards']), changeInterval))
    for ep in change_episodes:
        plt.axvline(x=ep, color='red', linestyle='--', alpha=0.5,
                   label='Environment Change' if ep == change_episodes[0] else None)
    
    # Add annotations for environment changes
    for i, ep in enumerate(change_episodes):
        change_type = "Leg Length" if i % 2 == 0 else "Terrain Roughness"
        plt.annotate(
            f"{change_type} Change",
            xy=(ep, plt.gca().get_ylim()[1] * 0.95),
            xytext=(ep + 50, plt.gca().get_ylim()[1] * 0.95),
            arrowprops=dict(facecolor='black', shrink=0.05, width=1.5, headwidth=8),
            fontsize=10,
            bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8)
        )
    
    # Add title and labels
    title_suffix = " with Variance Bands" if include_variance else ""
    plt.title(f'BipedalWalker Reward Performance Comparison{title_suffix}', fontsize=16)
    plt.xlabel('Episode', fontsize=14)
    plt.ylabel('Average Reward (smoothed)', fontsize=14)
    plt.grid(True, alpha=0.3)
    
    # Create a custom legend with larger markers
    plt.legend(fontsize=12, loc='upper center', bbox_to_anchor=(0.5, -0.15), 
               ncol=3, frameon=True, fancybox=True, shadow=True)
    
    # Adjust margins
    plt.tight_layout()
    
    # Save the plot if a path is provided
    if save_path:
        # Create the directory if it doesn't exist
        os.makedirs(save_path, exist_ok=True)
        
        # Use a descriptive filename
        variance_suffix = "_with_variance" if include_variance else ""
        filename = f"bipedal_reward_over_time{variance_suffix}.png"
        
        # Join the path correctly
        filepath = os.path.join(save_path, filename)
        
        # Print debug info
        print(f"Saving reward plot to: {filepath}")
        
        # Save with a good margin setting
        # Save and display approach
        plt.savefig(filepath, bbox_inches='tight', dpi=300)
        plt.show()
        plt.close()
        print(f"Saved plot to {filepath}")
    
    return plt.gcf()  # Return the figure object

def analyze_performance_breakdown(results, change_interval):
    """Analyze performance during different phases of the experiment"""
    
    # Identify environments
    env_changes = next(iter(results.values()))['environmentChanges']
    total_episodes = len(next(iter(results.values()))['rewards'])
    
    # Define environments/phases
    phases = []
    for i, change_ep in enumerate(env_changes):
        start_ep = 0 if i == 0 else env_changes[i-1]
        end_ep = change_ep
        
        # Add a phase
        phases.append({
            'name': f"Environment {i+1}",
            'start': start_ep,
            'end': end_ep,
            'length': end_ep - start_ep
        })
    
    # Add the final phase
    if env_changes:
        phases.append({
            'name': f"Environment {len(env_changes)+1}",
            'start': env_changes[-1],
            'end': total_episodes,
            'length': total_episodes - env_changes[-1]
        })
    
    # Calculate per-environment metrics
    phase_metrics = []
    
    for phase in phases:
        phase_data = {'name': phase['name']}
        
        for reward_type, reward_data in results.items():
            # Skip if this reward type doesn't have enough data
            if phase['end'] > len(reward_data['rewards']):
                continue
                
            # Calculate metrics for this phase
            rewards_slice = reward_data['rewards'][phase['start']:phase['end']]
            distance_slice = reward_data['distances'][phase['start']:phase['end']]
            
            # Skip empty slices
            if not rewards_slice or not distance_slice:
                continue
                
            phase_data[f"{reward_type}_avg_reward"] = np.mean(rewards_slice)
            phase_data[f"{reward_type}_avg_distance"] = np.mean(distance_slice)
            phase_data[f"{reward_type}_stability"] = 1.0 - (np.std(rewards_slice) / np.mean(rewards_slice)) if np.mean(rewards_slice) > 0 else 0
        
        phase_metrics.append(phase_data)
    
    # Convert to DataFrame for easier analysis
    phase_df = pd.DataFrame(phase_metrics)
    
    print("\nPerformance by Environment Phase:")
    print(phase_df.round(2))
    
    # Calculate relative performance (adaptive vs others)
    for phase in phase_metrics:
        # Skip if adaptive reward data is not available for this phase
        if 'adaptivereward_avg_reward' not in phase:
            continue
            
        for reward_type in [r for r in results.keys() if r != 'adaptivereward']:
            # Skip if this reward type doesn't have data for this phase
            if f"{reward_type}_avg_reward" not in phase:
                continue
                
            # Calculate relative performance
            relative_reward = (phase['adaptivereward_avg_reward'] / phase[f"{reward_type}_avg_reward"] - 1) * 100
            phase[f"relative_to_{reward_type}_pct"] = relative_reward
    
    # Create a visualization of performance by phase
    plt.figure(figsize=(14, 8))
    
    # Set up bar positions
    bar_width = 0.2
    index = np.arange(len(phase_metrics))
    
    # Plot bars for each reward type
    colors = ['b', 'g', 'r', 'c']
    for i, reward_type in enumerate(results.keys()):
        values = [phase.get(f"{reward_type}_avg_reward", 0) for phase in phase_metrics]
        plt.bar(index + i*bar_width, values, bar_width, 
               label=reward_type, color=colors[i], alpha=0.7)
    
    # Customize the plot
    plt.xlabel('Environment Phase')
    plt.ylabel('Average Reward')
    plt.title('BipedalWalker Performance Comparison Across Environment Phases')
    plt.xticks(index + bar_width, [phase['name'] for phase in phase_metrics])
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, "bipedal_performance_by_phase.png"), dpi=300, bbox_inches='tight')
    plt.show()
    
    # Create a heatmap of relative performance
    plt.figure(figsize=(12, 6))
    
    # Extract relative performance data
    relative_data = []
    for phase in phase_metrics:
        row = {'phase': phase['name']}
        for reward_type in [r for r in results.keys() if r != 'adaptivereward']:
            key = f"relative_to_{reward_type}_pct"
            if key in phase:
                row[reward_type] = phase[key]
        relative_data.append(row)
    
    # Convert to DataFrame
    relative_df = pd.DataFrame(relative_data).set_index('phase')
    
    # Create heatmap
    ax = sns.heatmap(relative_df, annot=True, cmap='RdYlGn', center=0, 
                    fmt='.1f', cbar_kws={'label': 'Relative Performance (%)'}, 
                    linewidths=0.5)
    
    plt.title('BipedalWalker Adaptive Reward Performance Relative to Other Approaches (%)')
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, "bipedal_relative_performance_heatmap.png"), dpi=300, bbox_inches='tight')
    plt.show()
    
    return phase_metrics

def analyze_adaptive_components(results, save_path="."):
    """Analyze the evolution and contribution of adaptive reward components"""
    
    # Check if adaptive reward results exist and have component data
    if 'adaptivereward' not in results or 'component_weights' not in results['adaptivereward']:
        print("No component data available for adaptive reward")
        return None
    
    # Extract component weight data
    component_data = results['adaptivereward']['component_weights']
    episodes = [d['episode'] for d in component_data]
    balance_weights = [d['stability'] for d in component_data]
    progress_weights = [d['efficiency'] for d in component_data]
    
    # Extract reward changes and environment changes
    reward_changes = results['adaptivereward']['rewardChanges']
    env_changes = results['adaptivereward']['environmentChanges']
    
    # Create weight evolution visualization
    plt.figure(figsize=(14, 8))
    
    # Plot component weights
    plt.plot(episodes, balance_weights, 'b-', label='Balance Weight', linewidth=2)
    plt.plot(episodes, progress_weights, 'g-', label='Progress Weight', linewidth=2)
    
    # Add vertical lines for reward function updates
    for ep in reward_changes:
        plt.axvline(x=ep, color='g', linestyle='--', alpha=0.5, 
                   label='Reward Update' if reward_changes.index(ep) == 0 else None)
    
    # Add vertical lines for environment changes
    for ep in env_changes:
        plt.axvline(x=ep, color='r', linestyle='--', alpha=0.5,
                   label='Environment Change' if env_changes.index(ep) == 0 else None)
    
    # Customize plot
    plt.title('Evolution of BipedalWalker Adaptive Reward Component Weights')
    plt.xlabel('Episode')
    plt.ylabel('Component Weight')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, "bipedal_component_weight_evolution.png"), dpi=300, bbox_inches='tight')
    plt.show()
    
    # Analyze performance correlation with weights
    rewards = results['adaptivereward']['rewards']
    
    # Create a DataFrame with all data
    df = pd.DataFrame({
        'episode': episodes,
        'balance_weight': balance_weights,
        'progress_weight': progress_weights,
        'reward': [rewards[ep] if ep < len(rewards) else np.nan for ep in episodes]
    })
    
    # Calculate rolling reward for smoother analysis
    df['rolling_reward'] = df['reward'].rolling(window=20).mean()
    
    # Calculate correlations
    corr_balance = df['balance_weight'].corr(df['rolling_reward'])
    corr_progress = df['progress_weight'].corr(df['rolling_reward'])
    
    print(f"\nCorrelation between balance weight and reward: {corr_balance:.3f}")
    print(f"Correlation between progress weight and reward: {corr_progress:.3f}")
    
    # Create a scatter plot of weights vs performance
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot balance weight vs reward
    sns.regplot(x='balance_weight', y='rolling_reward', data=df.dropna(), 
               ax=ax1, scatter_kws={'alpha': 0.3}, line_kws={'color': 'red'})
    ax1.set_title(f'Balance Weight vs Reward (corr = {corr_balance:.3f})')
    ax1.set_xlabel('Balance Weight')
    ax1.set_ylabel('Rolling Average Reward')
    ax1.grid(True, alpha=0.3)
    
    # Plot progress weight vs reward
    sns.regplot(x='progress_weight', y='rolling_reward', data=df.dropna(), 
               ax=ax2, scatter_kws={'alpha': 0.3}, line_kws={'color': 'red'})
    ax2.set_title(f'Progress Weight vs Reward (corr = {corr_progress:.3f})')
    ax2.set_xlabel('Progress Weight')
    ax2.set_ylabel('Rolling Average Reward')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, "bipedal_component_weight_correlation.png"), dpi=300, bbox_inches='tight')
    plt.show()
    
    # Analyze component updates if available
    if 'component_updates' in results['adaptivereward'] and results['adaptivereward']['component_updates']:
        updates = results['adaptivereward']['component_updates']
        
        print("\nComponent Update Analysis:")
        print(f"Total updates: {len(updates)}")
        
        # Count updates by component
        component_counts = {}
        for update in updates:
            component = f"Component {update['component']}"
            if component not in component_counts:
                component_counts[component] = 0
            component_counts[component] += 1
        
        for component, count in component_counts.items():
            print(f"{component}: {count} updates")
    
    return df

def analyze_adaptation_speed(results, change_interval, save_path="."):
    """
    Analyze how quickly each reward approach adapts to environment changes.
    """
    adaptation_metrics = {}
    
    for reward_type, reward_data in results.items():
        # Skip if no rewards data
        if 'rewards' not in reward_data or not reward_data['rewards']:
            continue
            
        rewards = reward_data['rewards']
        env_changes = reward_data['environmentChanges']
        
        # Skip if no environment changes
        if not env_changes:
            continue
            
        # Calculate metrics for each change
        recovery_times = []
        performance_drops = []
        adaptation_effectiveness = []
        
        for change_ep in env_changes:
            # Skip if change is too close to the end
            if change_ep + 100 >= len(rewards):
                continue
                
            # Calculate pre-change performance (20 episodes before change)
            pre_change_start = max(0, change_ep - 20)
            pre_change_rewards = rewards[pre_change_start:change_ep]
            pre_change_avg = np.mean(pre_change_rewards) if pre_change_rewards else 0
            
            # Calculate immediate post-change performance (20 episodes after change)
            post_change_rewards = rewards[change_ep:change_ep+20]
            post_change_avg = np.mean(post_change_rewards) if post_change_rewards else 0
            
            # Calculate performance drop
            if pre_change_avg > 0:
                drop_pct = max(0, (pre_change_avg - post_change_avg) / pre_change_avg * 100)
                performance_drops.append(drop_pct)
            
            # Calculate recovery time
            recovery_threshold = 0.9 * pre_change_avg  # 90% of pre-change performance
            recovery_ep = change_ep + change_interval  # Default: never recovered
            
            for i in range(change_ep, min(change_ep + change_interval, len(rewards))):
                # Use sliding window of 10 episodes
                window_start = max(change_ep, i - 10)
                window_rewards = rewards[window_start:i+1]
                window_avg = np.mean(window_rewards) if window_rewards else 0
                
                if window_avg >= recovery_threshold:
                    recovery_ep = i
                    break
            
            recovery_time = recovery_ep - change_ep
            recovery_times.append(recovery_time)
            
            # Calculate adaptation effectiveness
            if recovery_time > 0 and change_ep + recovery_time < len(rewards):
                # Area under the recovery curve
                recovery_rewards = rewards[change_ep:change_ep + recovery_time]
                actual_area = np.sum(recovery_rewards)
                ideal_area = pre_change_avg * recovery_time
                
                if ideal_area > 0:
                    effectiveness = actual_area / ideal_area
                    adaptation_effectiveness.append(effectiveness)
        
        # Store metrics for this reward type
        adaptation_metrics[reward_type] = {
            'avg_recovery_time': np.mean(recovery_times) if recovery_times else float('nan'),
            'avg_performance_drop': np.mean(performance_drops) if performance_drops else float('nan'),
            'avg_adaptation_effectiveness': np.mean(adaptation_effectiveness) if adaptation_effectiveness else float('nan'),
            'recovery_times': recovery_times,
            'performance_drops': performance_drops
        }
    
    # Display results
    print("\nBipedalWalker Adaptation Speed Analysis:")
    for reward_type, metrics in adaptation_metrics.items():
        print(f"\n{reward_type}:")
        print(f"  Average Recovery Time: {metrics['avg_recovery_time']:.2f} episodes")
        print(f"  Average Performance Drop: {metrics['avg_performance_drop']:.2f}%")
        print(f"  Adaptation Effectiveness: {metrics['avg_adaptation_effectiveness']:.2f}")
    
    # Create visualization: Recovery Time Comparison
    plt.figure(figsize=(12, 6))
    
    labels = list(adaptation_metrics.keys())
    recovery_times = [m['avg_recovery_time'] for m in adaptation_metrics.values()]
    performance_drops = [m['avg_performance_drop'] for m in adaptation_metrics.values()]
    
    x = np.arange(len(labels))
    width = 0.35
    
    fig, ax1 = plt.subplots(figsize=(12, 6))
    
    # Plot recovery times
    bars1 = ax1.bar(x - width/2, recovery_times, width, label='Recovery Time (episodes)', color='b', alpha=0.7)
    ax1.set_ylabel('Recovery Time (episodes)', color='b')
    ax1.tick_params(axis='y', colors='b')
    
    # Add second y-axis for performance drop
    ax2 = ax1.twinx()
    bars2 = ax2.bar(x + width/2, performance_drops, width, label='Performance Drop (%)', color='r', alpha=0.7)
    ax2.set_ylabel('Performance Drop (%)', color='r')
    ax2.tick_params(axis='y', colors='r')
    
    # Add labels and legend
    ax1.set_title('BipedalWalker Adaptation Speed Comparison')
    ax1.set_xticks(x)
    ax1.set_xticklabels(labels)
    ax1.set_xlabel('Reward Approach')
    
    # Add value labels on bars
    for i, bar in enumerate(bars1):
        height = bar.get_height()
        if not np.isnan(height):
            ax1.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                    f'{height:.1f}', ha='center', va='bottom', color='b', fontweight='bold')
    
    for i, bar in enumerate(bars2):
        height = bar.get_height()
        if not np.isnan(height):
            ax2.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                    f'{height:.1f}%', ha='center', va='bottom', color='r', fontweight='bold')
    
    # Combine legends
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, "bipedal_adaptation_speed_comparison.png"), dpi=300, bbox_inches='tight')
    plt.show()
    
    # Recovery Curves Visualization
    plt.figure(figsize=(14, 8))
    
    # Get the first environment change episode
    env_changes = next(iter(results.values()))['environmentChanges']
    if not env_changes:
        return adaptation_metrics
    
    first_change = env_changes[0]
    window = 100  # Episodes to show after change
    
    # Plot recovery curves for each approach
    for reward_type, reward_data in results.items():
        if first_change >= len(reward_data['rewards']):
            continue
            
        # Get pre-change performance as baseline
        pre_change_start = max(0, first_change - 20)
        pre_change_rewards = reward_data['rewards'][pre_change_start:first_change]
        pre_change_avg = np.mean(pre_change_rewards) if pre_change_rewards else 1.0
        
        # Get post-change rewards
        post_window = min(window, len(reward_data['rewards']) - first_change)
        post_rewards = reward_data['rewards'][first_change:first_change + post_window]
        
        # Normalize as percentage of pre-change performance
        normalized_rewards = [r / pre_change_avg * 100 for r in post_rewards]
        
        # Smooth the curve
        smoothed = gaussian_filter1d(normalized_rewards, sigma=2)
        
        # Plot the recovery curve
        plt.plot(range(len(smoothed)), smoothed, label=reward_type, linewidth=2)
    
    # Add pre-change level reference line
    plt.axhline(y=100, color='k', linestyle='--', alpha=0.5, label='Pre-change level')
    
    # Add labels and legend
    plt.title('BipedalWalker Recovery Curves After Environment Change')
    plt.xlabel('Episodes After Change')
    plt.ylabel('Performance (% of pre-change)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, "bipedal_recovery_curves.png"), dpi=300, bbox_inches='tight')
    plt.show()
    
    return adaptation_metrics

In [ ]:
def run_complete_bipedal_analysis(episodes=5000, change_interval=1000, num_runs=1):
    """Run a complete analysis of BipedalWalker performance with all metrics"""
    all_results = []
    
    # Create results directory
    main_results_folder = "BipedalPerformanceResults"
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    experiment_folder = os.path.join(main_results_folder, f"BipedalAnalysis_{timestamp}")
    os.makedirs(experiment_folder, exist_ok=True)
    
    for run in range(num_runs):
        print(f"\n=============== Starting Run {run+1}/{num_runs} ===============")
        
        # Run the performance test
        seed = 42 + run
        results = runBipedalPerformanceTest(
            episodes=episodes,
            changeInterval=change_interval,
            legChanges=[0.3, 0.5],
            terrainChanges=[0.0, 1.0],
            seed=seed,
            collect_component_data=True
        )
        
        all_results.append(results)
        
        # Create a subfolder for this run
        run_folder = os.path.join(experiment_folder, f"run_{run+1}")
        os.makedirs(run_folder, exist_ok=True)
        
        # Run analyses for this iteration
        print("\n--- Running Statistical Significance Tests ---")
        statistical_results = run_statistical_tests(results)
        
        print("\n--- Analyzing Adaptation Speed ---")
        adaptation_metrics = analyze_adaptation_speed(results, change_interval, save_path=run_folder)
        
        print("\n--- Analyzing Performance Breakdown ---")
        phase_metrics = analyze_performance_breakdown(results, change_interval)
        
        print("\n--- Analyzing Adaptive Components ---")
        component_analysis = analyze_adaptive_components(results, save_path=run_folder)
        
        # Create reward over time plots
        print("\n--- Creating Reward Over Time Plots ---")
        reward_plot_with_variance = create_reward_over_time_plot(
            results, 
            change_interval, 
            save_path=run_folder,
            include_variance=True
        )
        plt.close(reward_plot_with_variance)
        
        reward_plot_without_variance = create_reward_over_time_plot(
            results, 
            change_interval, 
            save_path=run_folder,
            include_variance=False
        )
        plt.close(reward_plot_without_variance)
        
        # Save results to JSON
        analysis_summary = {
            'statistical_results': {
                metric: {
                    'anova_p_value': stats['anova_p_value'],
                    'anova_significant': stats['anova_significant'],
                    'pairwise_significant': {
                        pair: result['significant'] 
                        for pair, result in stats['pairwise_comparisons'].items()
                    }
                }
                for metric, stats in statistical_results.items()
            },
            'adaptation_metrics': {
                reward_type: {
                    'avg_recovery_time': metrics['avg_recovery_time'],
                    'avg_performance_drop': metrics['avg_performance_drop']
                }
                for reward_type, metrics in adaptation_metrics.items()
            },
            'phase_metrics': [{
                'name': phase['name'],
                'best_approach': max([
                    (reward_type, phase.get(f"{reward_type}_avg_reward", 0))
                    for reward_type in results.keys()
                ], key=lambda x: x[1])[0] if phase else None
            } for phase in phase_metrics]
        }
        
        # Convert numpy types to Python types for JSON serialization
        def convert_to_python_types(obj):
            if isinstance(obj, np.floating):
                return float(obj)
            elif isinstance(obj, np.integer):
                return int(obj)
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            elif isinstance(obj, dict):
                return {k: convert_to_python_types(v) for k, v in obj.items()}
            elif isinstance(obj, list):
                return [convert_to_python_types(item) for item in obj]
            else:
                return obj
        
        with open(os.path.join(run_folder, 'bipedal_analysis_summary.json'), 'w') as f:
            json.dump(convert_to_python_types(analysis_summary), f, indent=2)
    
    # If multiple runs, aggregate results
    if num_runs > 1:
        print("\n=============== Aggregating Results Across Runs ===============")
        
        # Create aggregate reward plot
        plt.figure(figsize=(16, 10))
        
        # Define colors and line styles
        colors = {
            'adaptivereward': '#1f77b4',  # blue
            'energy_based': '#2ca02c',    # green
            'pbrs': '#ff7f0e',            # orange
            'baseline': '#d62728'         # red
        }
        
        line_styles = {
            'adaptivereward': '-',
            'energy_based': '-.',
            'pbrs': '--',
            'baseline': ':'
        }
        
        # Organize data by reward type
        reward_data = {}
        max_episodes = 0
        
        # Collect data from all runs
        for run_results in all_results:
            for reward_type, results in run_results.items():
                if reward_type not in reward_data:
                    reward_data[reward_type] = []
                
                # Get reward data with smoothing
                rewards = pd.Series(results['rewards']).rolling(window=100, min_periods=1).mean()
                reward_data[reward_type].append(rewards)
                max_episodes = max(max_episodes, len(rewards))
        
        # Plot each reward type with confidence interval
        for reward_type, runs in reward_data.items():
            # Make sure all runs have the same length
            padded_runs = []
            for run in runs:
                if len(run) < max_episodes:
                    padded = run.reindex(range(max_episodes), fill_value=np.nan)
                else:
                    padded = run
                padded_runs.append(padded)
            
            # Convert to numpy array
            run_data = np.array([run.values for run in padded_runs])
            
            # Calculate mean and confidence intervals
            mean_rewards = np.nanmean(run_data, axis=0)
            if run_data.shape[0] > 1:
                std_rewards = np.nanstd(run_data, axis=0)
                ci = 1.96 * std_rewards / np.sqrt(run_data.shape[0])  # 95% CI
            else:
                ci = np.zeros_like(mean_rewards)
            
            # Plot mean line
            plt.plot(
                range(len(mean_rewards)),
                mean_rewards,
                label=reward_type,
                color=colors.get(reward_type, 'black'),
                linestyle=line_styles.get(reward_type, '-'),
                linewidth=3
            )
            
            # Plot confidence interval
            plt.fill_between(
                range(len(mean_rewards)),
                mean_rewards - ci,
                mean_rewards + ci,
                color=colors.get(reward_type, 'black'),
                alpha=0.2
            )
        
        # Add vertical lines for environment changes
        change_episodes = list(range(change_interval, max_episodes, change_interval))
        for i, ep in enumerate(change_episodes):
            # Skip if beyond our data
            if ep >= max_episodes:
                continue
                
            plt.axvline(
                x=ep,
                color='black',
                linestyle='--',
                alpha=0.5,
                label='Environment Change' if i == 0 else None
            )
            
            # Add annotation
            change_type = "Leg Length" if i % 2 == 0 else "Terrain Roughness"
            y_pos = plt.gca().get_ylim()[1] * 0.95
            plt.annotate(
                f"{change_type} Change",
                xy=(ep, y_pos),
                xytext=(ep + max_episodes * 0.02, y_pos),
                arrowprops=dict(facecolor='black', shrink=0.05, width=1.5, headwidth=8),
                fontsize=12,
                bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8)
            )
        
        # Add title and labels
        plt.title('BipedalWalker Aggregate Reward Performance Across All Runs (with 95% CI)', fontsize=18)
        plt.xlabel('Episode', fontsize=16)
        plt.ylabel('Average Reward (smoothed)', fontsize=16)
        plt.grid(True, alpha=0.3)
        
        # Create a custom legend
        plt.legend(
            fontsize=14,
            loc='upper center',
            bbox_to_anchor=(0.5, -0.1),
            ncol=4,
            frameon=True,
            fancybox=True,
            shadow=True
        )
        
        # Adjust margins
        plt.tight_layout()
        
        # Save the plot
        filepath = os.path.join(experiment_folder, "bipedal_aggregate_reward_comparison.png")
        plt.savefig(filepath, bbox_inches='tight', dpi=300)
        plt.show()
        print(f"Saved aggregate plot: bipedal_aggregate_reward_comparison.png in {experiment_folder}")
        plt.close()
    
    print(f"\nAnalysis complete. Results saved in {experiment_folder}")
    return all_results

In [ ]:
# Run a complete analysis with smaller episodes for testing
episodes = 50000
change_interval = 25000
num_runs = 2  # Set to more runs for better statistical power

print(f"Running complete BipedalWalker analysis with {num_runs} runs, {episodes} episodes and change interval {change_interval}")

# Run the complete analysis
results = run_complete_bipedal_analysis(
    episodes=episodes, 
    change_interval=change_interval, 
    num_runs=num_runs
)

print("\nAnalysis complete - All functions have been executed:")
print("✓ run_statistical_tests")
print("✓ analyze_adaptation_speed")
print("✓ analyze_performance_breakdown")
print("✓ analyze_adaptive_components")
print("✓ create_reward_over_time_plot")

In [ ]:
"""
## Conclusion and Key Findings for BipedalWalker Environment

This analysis has provided detailed insights into the performance of adaptive reward functions in the 
more complex BipedalWalker environment:

1. **Statistical Significance**: The differences between reward approaches were [significant/not significant] 
   with p-values of [values from your test results].

2. **Adaptation Speed**: Adaptive reward functions demonstrated [faster/slower] recovery times after 
   environmental changes, with average recovery times of [X episodes] compared to [Y episodes] for 
   static approaches.

3. **Performance Analysis**: 
   - Adaptive rewards showed particularly strong performance in [which environments]
   - The terrain roughness changes had [more/less] impact than leg length changes
   - Performance stabilized [quickly/slowly] after disruptions

4. **Component Mechanism**: 
   - The adaptive approach adjusted component weights with [pattern observed]
   - Balance components dominated in [which conditions]
   - Progress components became more important in [which conditions]
   - Component updates showed [what pattern] with environmental changes

5. **Comparison to CartPole**:
   - The BipedalWalker environment showed [similarities/differences] in adaptation patterns
   - Reward function adaptation was [more/less] effective in this more complex environment
   - Recovery times were [longer/shorter] than in CartPole experiments

These findings [support/challenge] our hypothesis about adaptive reward functions in complex environments.
The BipedalWalker experiments demonstrate that our approach [generalizes/struggles] when applied to 
higher-dimensional state and action spaces.
"""

# Create a summary table of key metrics to include in your paper
import pandas as pd
from IPython.display import display, HTML

# Extract key metrics from results
metrics_data = []
for reward_type in ['adaptivereward', 'energy_based', 'baseline', 'pbrs']:
    if reward_type in results[0]:
        row = {
            'Reward Approach': reward_type,
            'Avg Reward': np.mean(results[0][reward_type]['rewards'][-100:]),
            'Avg Distance': np.mean(results[0][reward_type]['distances'][-100:]),
            'Recovery Time (eps)': results[0][reward_type]['adaptation_metrics'].get('avg_recovery_time', 'N/A'),
            'Perf. Drop (%)': results[0][reward_type]['adaptation_metrics'].get('avg_performance_drop', 'N/A'),
        }
        metrics_data.append(row)

summary_df = pd.DataFrame(metrics_data)
display(HTML(summary_df.to_html(index=False)))

# Also print in markdown format for easy inclusion in paper
print("\nMarkdown Table for Paper:\n")
print(summary_df.to_markdown(index=False))